 ### Installing all the necessary libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

### Then importing all the needed libraries

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, Trainer, pipeline
from peft import LoraConfig
from datasets import Dataset
from langchain.prompts.prompt import PromptTemplate

from trl import SFTTrainer
from peft import PeftModel

import pandas as pd

In [ ]:
#model_name="meta-llama/CodeLlama-7b-hf" #base model that u want to finetune
model_name="meta-llama/Llama-2-7b-hf"

### Quantization

In [ ]:
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)
#tokenizer from any model can be taken
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### We need to be careful and thorough about the data we use to finetune the model , so we take the csv data and we format it in a way that it gets fed to the modal in template form

In [ ]:
df_data = pd.read_excel("/content/Gen AI Questions nish.xlsx")

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["category", "question", "SQL_Queries"], template="<s>[INST]<<SYS>>You are an agent designed to interact with a SQL database.Given an input question, create a syntactically correct teradatasql query to run. This is a {category} question. </SYS>{question}[/INST]{sql_query}</s>"
)

prompt_data = []
for i, row in df_data.iterrows():
  prompt_data.append(prompt_template.format(category=row.category, question=row.question, sql_query=row.SQL_Queries))

dataset = Dataset.from_dict({"inputs": prompt_data})

In [ ]:
print(dataset[2].get('inputs'))

<s>[INST]<<SYS>>You are an agent designed to interact with a SQL database.Given an input question, create a syntactically correct teradatasql query to run. This is a Bank Balances question. </SYS>What is my currency exposure based on my bank balances in all my bank accounts?[/INST]SELECT currency_code, SUM((balance->>'available')::numeric) AS total_balance FROM bank_accounts b GROUP BY currency_code ORDER BY total_balance DESC</s>


In [ ]:
dataset.to_json("./data/finetune_data.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

27362

### from here on, the training arguments settings and training starts:-

### This is for finding out what are the linear layers of our large language model so that we can specify those layers for finetuning

In [ ]:
import bitsandbytes as bnb
# to find all the linear layers of the models so that we can specify them during finetuning
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            # model-specific
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)

In [ ]:
find_all_linear_names(model=model)

['down_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj', 'gate_proj']

In [ ]:

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_params = LoraConfig(
        lora_dropout=0.05,
        rank=64,
        lora_alpha=128,  # alpha's value should be two times the rank
        bias="none",
        target_modules=['down_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj', 'gate_proj'],
        task_type="CAUSAL_LM",
)

In [ ]:
# Set training parameters
training_params= TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
)


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="inputs",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=13, training_loss=1.4492940169114332, metrics={'train_runtime': 212.8724, 'train_samples_per_second': 0.244, 'train_steps_per_second': 0.061, 'total_flos': 214974533861376.0, 'train_loss': 1.4492940169114332, 'epoch': 1.0})

In [ ]:
new_model = "./drive/MyDrive/finetune_model/codellama_sql-sft-3" #ur model, change the name as u like

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('./drive/MyDrive/finetune_model/codellama_sql-sft-3/tokenizer_config.json',
 './drive/MyDrive/finetune_model/codellama_sql-sft-3/special_tokens_map.json',
 './drive/MyDrive/finetune_model/codellama_sql-sft-3/tokenizer.model',
 './drive/MyDrive/finetune_model/codellama_sql-sft-3/added_tokens.json',
 './drive/MyDrive/finetune_model/codellama_sql-sft-3/tokenizer.json')

### to save the merged model

In [ ]:
model_name ="meta-llama/Llama2-7b-hf"


In [ ]:
# 1. Load based model
base_model = AutoModelForCausalLM.from_pretrained(model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 

In [ ]:
new_model="/content/drive/MyDrive/finetune_model/codellama_sql-sft-2"

In [ ]:
# 2. Load new fine-tuned model. Then merge this two model
finetuned_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = finetuned_model.merge_and_unload()

In [ ]:
pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
               max_new_tokens=512
               )

In [ ]:
from huggingface_hub import login
login()

#### uploading merged model also


In [ ]:
merged_model.push_to_hub("llama2-sql-sft")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nishika26/codellama-sql-sft-merged-2/commit/1843f773241bc83e2abb635fe87e0fd9c9f3e89d', commit_message='Upload LlamaForCausalLM', commit_description='', oid='1843f773241bc83e2abb635fe87e0fd9c9f3e89d', pr_url=None, pr_revision=None, pr_num=None)

### to save just the lora adapters

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

NameError: name 'trainer' is not defined

In [ ]:
from peft import LoraConfig, get_peft_model

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
model.push_to_hub("llama2-sql-sft-adapters")